# 대스타

3차 모델구현

### Library

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model,Sequential
import zipfile
import pandas as pd

import gdown

### Data Download

In [2]:
google_path = 'https://drive.google.com/uc?id='
file_id = '1PwuXmL0HdmjsThBHDKhaCfKK3jvdoAIG'
output_name = 'train.zip'
gdown.download(google_path+file_id,output_name)

Downloading...
From: https://drive.google.com/uc?id=1PwuXmL0HdmjsThBHDKhaCfKK3jvdoAIG
To: /content/train.zip
100%|██████████| 1.37G/1.37G [00:14<00:00, 97.1MB/s]


'train.zip'

In [3]:
input_path = './train.zip'
output_path = './data'

In [4]:
zip_data = zipfile.ZipFile(input_path)
zip_data.extractall(output_path)

In [5]:
train_path = './data/output2/train'
val_path = './data/output2/val'
test_path = './data/output2/test'

label_path = './data/label.csv'

### 변수 선언

In [6]:
# 예측할 클래스 수 = len(df)
classes = 309

# Input으로 사용될 크기와 채널수
height = 256
width = 256
channels = 3

#클래스별 key:value 딕셔너리 저장
dic = {}

In [7]:
df = pd.read_csv(label_path, encoding='cp949')

df

,index,file_name
0,0,가는기린초
1,1,가막살나무
2,2,가문비나무
3,3,가시여뀌
4,4,가을강아지풀
...,...,...
304,304,환삼덩굴
305,305,황금
306,306,황새냉이
307,307,회목나무


In [8]:
for i in range(len(df)):
  dic[df.iloc[i][0]] = df.iloc[i][1]

In [9]:
dic

{0: '가는기린초',
 1: '가막살나무',
 2: '가문비나무',
 3: '가시여뀌',
 4: '가을강아지풀',
 5: '가침박달',
 6: '각시취',
 7: '감국',
 8: '개갓냉이',
 9: '개구리발톱',
 10: '개구리자리',
 11: '개기장',
 12: '개다래',
 13: '개똥쑥',
 14: '개병풍',
 15: '개쑥부쟁이',
 16: '개양귀비',
 17: '개여뀌',
 18: '개옻나무',
 19: '개찌버리사초',
 20: '갯패랭이꽃',
 21: '거제수나무',
 22: '고광나무',
 23: '고들빼기',
 24: '고려엉겅퀴',
 25: '고본',
 26: '고욤나무',
 27: '고추나무',
 28: '고추나물',
 29: '과남풀',
 30: '괴불나무',
 31: '구상나무',
 32: '구절초',
 33: '국수나무',
 34: '그늘흰사초',
 35: '그령',
 36: '금강아지풀',
 37: '금강초롱꽃',
 38: '금계국',
 39: '금꿩의다리',
 40: '금방동사니',
 41: '금불초',
 42: '기린초',
 43: '긴산꼬리풀',
 44: '까마귀머루',
 45: '까마귀밥나무',
 46: '까실쑥부쟁이',
 47: '까치고들빼기',
 48: '깽깽이풀',
 49: '꼬리조팝나무',
 50: '꼬리풀',
 51: '꽃다지',
 52: '꽃마리',
 53: '꽃며느리밥풀',
 54: '꽃범의꼬리',
 55: '꽃향유',
 56: '꽝꽝나무',
 57: '꿀풀',
 58: '끈끈이여뀌',
 59: '나도개피',
 60: '나도냉이',
 61: '나도바람꽃',
 62: '나도생강',
 63: '나래박쥐나물',
 64: '나래새',
 65: '낙지다리',
 66: '냉이',
 67: '냉초',
 68: '너도바람꽃',
 69: '네귀쓴풀',
 70: '노각나무',
 71: '노간주나무',
 72: '노란장대',
 73: '노루오줌',
 74: '노린재나무',
 75: '노박덩굴',
 76: '눈개승마',

In [10]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator()
image_data_train = image_generator.flow_from_directory(train_path)
image_data_val = image_generator.flow_from_directory(val_path)



Found 34461 images belonging to 309 classes.
Found 4207 images belonging to 309 classes.


### Model Load

In [11]:
resnetv2 = tf.keras.applications.ResNet50V2(include_top=False,input_shape=(height,width,channels))

94683136/94668760 [==============================] - 0s 0us/step


In [12]:
resnetv2.trainable=False

In [13]:
model = Sequential([
                 resnetv2,
                 Dense(512,activation='relu'),
                 BatchNormalization(),
                 GlobalAveragePooling2D(),
                 Dense(classes,activation='softmax')
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 8, 8, 2048)        23564800  
                                                                 
 dense (Dense)               (None, 8, 8, 512)         1049088   
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 512)        2048      
 ormalization)                                                   
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_1 (Dense)             (None, 309)               158517    
                                                                 
Total params: 24,774,453
Trainable params: 1,208,629
Non

In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [16]:
def scheduler(epoch, lr):
    if epoch < 2:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [17]:
lrs = tf.keras.callbacks.LearningRateScheduler(scheduler)

### Model Fit

In [18]:
model.fit(image_data_train,batch_size=64,epochs=10,callbacks=[lrs],validation_data=(image_data_val),
          validation_steps =image_data_val.samples/image_data_val.batch_size)

Epoch 1/10
1077/1077 [==============================] - 163s 139ms/step - loss: 4.1200 - accuracy: 0.1432 - val_loss: 3.7372 - val_accuracy: 0.1761 - lr: 0.0010
Epoch 2/10
1077/1077 [==============================] - 148s 137ms/step - loss: 2.7366 - accuracy: 0.3771 - val_loss: 2.6331 - val_accuracy: 0.3789 - lr: 0.0010
Epoch 3/10
1077/1077 [==============================] - 148s 137ms/step - loss: 1.9784 - accuracy: 0.5446 - val_loss: 2.1998 - val_accuracy: 0.4647 - lr: 9.0484e-04
Epoch 4/10
1077/1077 [==============================] - 148s 137ms/step - loss: 1.5430 - accuracy: 0.6417 - val_loss: 1.9382 - val_accuracy: 0.5362 - lr: 8.1873e-04
Epoch 5/10
1077/1077 [==============================] - 148s 137ms/step - loss: 1.2728 - accuracy: 0.7064 - val_loss: 1.5962 - val_accuracy: 0.6190 - lr: 7.4082e-04
Epoch 6/10
1077/1077 [==============================] - 148s 137ms/step - loss: 1.0734 - accuracy: 0.7533 - val_loss: 1.4448 - val_accuracy: 0.6570 - lr: 6.7032e-04
Epoch 7/10
1077/10

### Test

In [20]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import glob

In [21]:
img_height = 256
img_width = 256

In [22]:
tests = glob.glob('/content/data/output2/test/*')

In [23]:
tests[0].split('/')[-1]

'108'

In [24]:
for folder in tests:
  images = glob.glob(f'{folder}/*.jpg')
  for file in images:
    original = load_img(file, target_size = (img_height,img_width))

    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image , axis = 0)

    predict = np.argmax(model.predict(image_batch/255.))
    answer = int(folder.split('/')[-1])

    print(f'결과 : {dic[predict]} \t정답 : {dic[answer]}')

결과 : 고광나무 	정답 : 딱지꽃
결과 : 고광나무 	정답 : 딱지꽃
결과 : 풍차국 	정답 : 딱지꽃
결과 : 소사나무 	정답 : 딱지꽃
결과 : 소사나무 	정답 : 딱지꽃
결과 : 소사나무 	정답 : 딱지꽃
결과 : 고광나무 	정답 : 딱지꽃
결과 : 고광나무 	정답 : 딱지꽃
결과 : 고광나무 	정답 : 딱지꽃
결과 : 소사나무 	정답 : 딱지꽃
결과 : 소사나무 	정답 : 딱지꽃
결과 : 소사나무 	정답 : 딱지꽃
결과 : 고광나무 	정답 : 딱지꽃
결과 : 고광나무 	정답 : 딱지꽃
결과 : 고광나무 	정답 : 딱지꽃
결과 : 고광나무 	정답 : 딱지꽃
결과 : 고광나무 	정답 : 딱지꽃
결과 : 소사나무 	정답 : 딱지꽃
결과 : 소사나무 	정답 : 딱지꽃
결과 : 소사나무 	정답 : 산조팝나무
결과 : 고광나무 	정답 : 산조팝나무
결과 : 고광나무 	정답 : 산조팝나무
결과 : 고광나무 	정답 : 산조팝나무
결과 : 고광나무 	정답 : 산조팝나무
결과 : 고광나무 	정답 : 산조팝나무
결과 : 고광나무 	정답 : 산조팝나무
결과 : 고광나무 	정답 : 산조팝나무
결과 : 고광나무 	정답 : 산조팝나무
결과 : 고광나무 	정답 : 산조팝나무
결과 : 소사나무 	정답 : 산조팝나무
결과 : 소사나무 	정답 : 산조팝나무
결과 : 소사나무 	정답 : 오갈피나무
결과 : 개갓냉이 	정답 : 오갈피나무
결과 : 개갓냉이 	정답 : 오갈피나무
결과 : 고광나무 	정답 : 오갈피나무
결과 : 고광나무 	정답 : 오갈피나무
결과 : 소사나무 	정답 : 오갈피나무
결과 : 소사나무 	정답 : 오갈피나무
결과 : 소사나무 	정답 : 오갈피나무
결과 : 소사나무 	정답 : 오갈피나무
결과 : 소사나무 	정답 : 오갈피나무
결과 : 소사나무 	정답 : 오갈피나무
결과 : 개갓냉이 	정답 : 오갈피나무
결과 : 고광나무 	정답 : 꽃범의꼬리
결과 : 물양지꽃 	정답 : 꽃범의꼬리
결과 : 고광나무 	정답 : 꽃범의꼬리
결과 : 쑥부쟁이 	정답 : 꽃범의꼬리
결과 : 